<a href="https://colab.research.google.com/github/minsuk-heo/tf2/blob/master/jupyter_notebooks/10.Word2Vec_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentence Classification using LSTM and Pretrained Word2Vec

pretrain된 word2vec 모델과 LSTM을 활용환 텍스트 분류(긍/부정)

In [ ]:
from IPython.display import Image
Image(url= "https://raw.githubusercontent.com/minsuk-heo/tf2/master/img/10_train.png", width=800, height=300)

#from IPython.display import Image
#Image(url= "https://raw.githubusercontent.com/minsuk-heo/tf2/master/img/10_test.png", width=800, height=300)

In [ ]:
# pretrain 장점 : Train data에 없는 정보에 대해서도 예측가능(이미 더 큰 데이터로 학습됨.)

In [1]:
# If you want to understand the detail of Word2Vec, here is well explained video.
# from IPython.display import YouTubeVideo
# YouTubeVideo('64qSgA66P-8')
# YouTubeVideo('m2cg32uEeZw')

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  %tensorflow_version 2.x
except Exception:
  pass

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd

import tensorflow_hub as hub
import numpy as np

In [4]:
# Load Pretrained Word2Vec
embed = hub.load("https://tfhub.dev/google/Wiki-words-250/2")

In [5]:
def get_max_length(df):
    """
    - 최대 단어 수
    get max token counts from train data, 
    so we use this number as fixed length input to RNN cell
    """
    max_length = 0
    for row in df['review']:
        if len(row.split(" ")) > max_length:
            max_length = len(row.split(" "))
    return max_length

def get_word2vec_enc(reviews):
    """
    - word2vec 변환
    get word2vec value for each word in sentence.
    concatenate word in numpy array, so we can use it as RNN input
    """
    encoded_reviews = []
    for review in reviews:
        tokens = review.split(" ")
        word2vec_embedding = embed(tokens)
        encoded_reviews.append(word2vec_embedding)
    return encoded_reviews
        
def get_padded_encoded_reviews(encoded_reviews):
    """
    - zero padding
    for short sentences, we prepend zero padding so all input to RNN has same length
    """
    padded_reviews_encoding = []
    for enc_review in encoded_reviews:
        zero_padding_cnt = max_length - enc_review.shape[0]
        pad = np.zeros((1, 250))
        for i in range(zero_padding_cnt):
            enc_review = np.concatenate((pad, enc_review), axis=0)
        padded_reviews_encoding.append(enc_review)
    return padded_reviews_encoding

def sentiment_encode(sentiment):
    """
    - 긍/부정 endoding
    return one hot encoding for Y value
    """
    if sentiment == 'positive':
        return [1,0]
    else:
        return [0,1]
    
def preprocess(df):
    """
    encode text value to numeric value
    """
    # encode words into word2vec
    reviews = df['review'].tolist()
    
    encoded_reviews = get_word2vec_enc(reviews)
    padded_encoded_reviews = get_padded_encoded_reviews(encoded_reviews)
    # encoded sentiment
    sentiments = df['sentiment'].tolist()
    encoded_sentiment = [sentiment_encode(sentiment) for sentiment in sentiments]
    X = np.array(padded_encoded_reviews)
    Y = np.array(encoded_sentiment)
    return X, Y 

# Preprocess (encode text to number)

In [6]:
movie_reviews_train = [
         {'review': 'this is the best movie', 'sentiment': 'positive'},
         {'review': 'i recommend you watch this movie', 'sentiment': 'positive'},
         {'review': 'it was waste of money and time', 'sentiment': 'negative'},
         {'review': 'the worst movie ever', 'sentiment': 'negative'}
    ]
df = pd.DataFrame(movie_reviews_train)

# LSTM의 INPUT값이 고정값이기 때문에 MAX LENGTH 기준으로 일치 필요
max_length = get_max_length(df)

train_X, train_Y = preprocess(df)

# Build Model

In [7]:
# LSTM model
model = Sequential()
model.add(LSTM(32))
model.add(Dense(2, activation='softmax')) # 긍/부정 분류

In [8]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# Train

In [9]:
print('Train...')
model.fit(train_X, train_Y,epochs=50)

Train...
Epoch 1/50
1/1 [==============================] - 5s 5s/step - loss: 0.6994 - accuracy: 0.5000
Epoch 2/50
1/1 [==============================] - 0s 11ms/step - loss: 0.6890 - accuracy: 0.5000
Epoch 3/50
1/1 [==============================] - 0s 16ms/step - loss: 0.6787 - accuracy: 1.0000
Epoch 4/50
1/1 [==============================] - 0s 16ms/step - loss: 0.6687 - accuracy: 1.0000
Epoch 5/50
1/1 [==============================] - 0s 13ms/step - loss: 0.6587 - accuracy: 1.0000
Epoch 6/50
1/1 [==============================] - 0s 13ms/step - loss: 0.6487 - accuracy: 1.0000
Epoch 7/50
1/1 [==============================] - 0s 12ms/step - loss: 0.6387 - accuracy: 1.0000
Epoch 8/50
1/1 [==============================] - 0s 9ms/step - loss: 0.6284 - accuracy: 1.0000
Epoch 9/50
1/1 [==============================] - 0s 20ms/step - loss: 0.6180 - accuracy: 1.0000
Epoch 10/50
1/1 [==============================] - 0s 10ms/step - loss: 0.6072 - accuracy: 1.0000
Epoch 11/50
1/1 [======

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 32)                36224     
                                                                 
 dense (Dense)               (None, 2)                 66        
                                                                 
Total params: 36,290
Trainable params: 36,290
Non-trainable params: 0
_________________________________________________________________


In [11]:
# Test 데이터의 nice, better 단어가 train에 없지만 best와 유사한 벡터값을 가지고 있으므로 예측가능(word2vec이 pretrain된 데이터 이므로.)

In [12]:
"""
movie_reviews_train = [
         {'review': 'this is the best movie', 'sentiment': 'positive'},
         {'review': 'i recommend you watch this movie', 'sentiment': 'positive'},
         {'review': 'it was waste of money and time', 'sentiment': 'negative'},
         {'review': 'the worst movie ever', 'sentiment': 'negative'}
    ]
"""
movie_reviews_test = [
         {'review': 'it is better movie', 'sentiment': 'positive'},
         {'review': 'i suggest you see this movie', 'sentiment': 'positive'},
         {'review': 'it was just throwing 20 dollars away', 'sentiment': 'negative'},
         {'review': 'worse than any show', 'sentiment': 'negative'},
         {'review': 'nice movie, so love it', 'sentiment': 'positive'},
         {'review': 'terrible', 'sentiment': 'negative'}
    ]
test_df = pd.DataFrame(movie_reviews_test)

test_X, test_Y = preprocess(test_df)

score, acc = model.evaluate(test_X, test_Y, verbose=2)
print('Test score:', score)
print('Test accuracy:', acc)

1/1 - 1s - loss: 0.2012 - accuracy: 1.0000 - 1s/epoch - 1s/step
Test score: 0.2012208253145218
Test accuracy: 1.0
